# Test Dask with Parquet Files

1. Serial
2. `dask.distributed`

In [1]:
import os

import numpy as np

from bokeh.io import output_notebook

import dask as da
import pandas as pd
import dask.dataframe as dd

In [2]:
tract = 4850

data_dir = '/global/projecta/projectdirs/lsst/global/in2p3/Run1.1/summary'

datafile_tract = os.path.join(data_dir, 'dpdd_object_tract_%d.parquet' % tract)
datafile_all = os.path.join(data_dir, 'dpdd_object.parquet')

In [3]:
from dask.distributed import Client
client = Client(processes=False)

In [13]:
# Switch between these to go from analyzing one tract ('datafile_tract'), which renders in seconds
# to the entire set of 20 tracts ('datafile_all'), which renders in minutes.
datafile = datafile_tract
# datafile = datafile_all

# Specify the columns we need.  This allows for significant performance advantages when reading a column-based storage format such as Parquet.
columns_to_read = ['mag_g', 'mag_r']
da_df = dd.read_parquet(datafile, columns=columns_to_read)
da_df_all = dd.read_parquet(datafile_all, columns=columns_to_read)

In [14]:
df2 = np.mean(da_df['mag_g'] - da_df['mag_r'])
df2_all = np.mean(da_df_all['mag_g'] - da_df_all['mag_r'])

In [15]:
%%timeit
df2.compute()

1.02 s ± 19.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit
df2_all.compute()

13.8 s ± 2.97 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
# Load Bokeh into the Notebook
output_notebook()

Loading BokehJS ...

In [18]:
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler, visualize

In [19]:
with da.config.set(scheduler='dask.distributed'), \
        Profiler() as prof, \
        ResourceProfiler(dt=0.25) as rprof, \
        CacheProfiler() as cprof:
    result = df2_all.compute()

In [20]:
visualize([prof, rprof, cprof])

bokeh.models.layouts.Column(
    id='278f4f8d-d950-4963-ab4b-882ef3371ca8',
    children=[bokeh.models.tools.ToolbarBox(
         id='c48a411b-4776-46bb-bb78-8ffcac4b176c',
         css_classes=[],
         disabled=False,
         height=None,
         js_event_callbacks={},
         js_property_callbacks={},
         name=None,
         sizing_mode='fixed',
         subscribed_events=[],
         tags=[],
         toolbar=bokeh.models.tools.ProxyToolbar(
             id='bad71fa3-24db-41a6-8ad1-d88102a35d74',
             js_event_callbacks={},
             js_property_callbacks={},
             logo='normal',
             name=None,
             subscribed_events=[],
             tags=[],
             tools=[bokeh.models.tools.HoverTool(
                  id='ee824a34-5ac4-43a6-9306-439024ae18b7',
                  anchor='center',
                  attachment='horizontal',
                  callback=None,
                  formatters={},
                  js_event_callbacks={},
                  js_property_callbacks={},
                  line_policy='nearest',
                  mode='mouse',
                  name=None,
                  names=[],
                  point_policy='follow_mouse',
                  renderers='auto',
                  show_arrow=True,
                  subscribed_events=[],
                  tags=[],
                  toggleable=True,
                  tooltips='\n    <div>\n        <span style="font-size: 14px; font-weight: bold;">Key:</span>&nbsp;\n        <span style="font-size: 10px; font-family: Monaco, monospace;">@key</span>\n    </div>\n    <div>\n        <span style="font-size: 14px; font-weight: bold;">Task:</span>&nbsp;\n        <span style="font-size: 10px; font-family: Monaco, monospace;">@function</span>\n    </div>\n    '),
              bokeh.models.tools.SaveTool(
                  id='c654014f-c07f-45fd-8305-599d8d72f6d9',
                  js_event_callbacks={},
                  js_property_callbacks={},
                  name=None,
                  subscribed_events=[],
                  tags=[]),
              bokeh.models.tools.ResetTool(
                  id='e858185c-dff7-4d24-9c00-7304dad4485f',
                  js_event_callbacks={},
                  js_property_callbacks={},
                  name=None,
                  subscribed_events=[],
                  tags=[]),
              bokeh.models.tools.WheelZoomTool(
                  id='10221567-934d-46bb-8296-3775fa9bff89',
                  dimensions='width',
                  js_event_callbacks={},
                  js_property_callbacks={},
                  maintain_focus=True,
                  name=None,
                  speed=0.0016666666666666668,
                  subscribed_events=[],
                  tags=[],
                  zoom_on_axis=True),
              bokeh.models.tools.PanTool(
                  id='87f2ac84-f22a-465c-abda-d5cb7278526f',
                  dimensions='width',
                  js_event_callbacks={},
                  js_property_callbacks={},
                  name=None,
                  subscribed_events=[],
                  tags=[]),
              bokeh.models.tools.SaveTool(
                  id='8b9d11f3-5ff3-4ccb-9ece-880e5a6c5e37',
                  js_event_callbacks={},
                  js_property_callbacks={},
                  name=None,
                  subscribed_events=[],
                  tags=[]),
              bokeh.models.tools.ResetTool(
                  id='ba682503-7d3a-4e25-9ff7-9951fd82e951',
                  js_event_callbacks={},
                  js_property_callbacks={},
                  name=None,
                  subscribed_events=[],
                  tags=[]),
              bokeh.models.tools.WheelZoomTool(
                  id='43c46186-8e9e-4208-a1af-42eaaf007a79',
                  dimensions='width',
                  js_event_callbacks={},
                  js_property_callbacks={},
                  maintain_focus=True,
     

In [21]:
%%timeit
with da.config.set(scheduler='dask.distributed'):
    result = df2_all.compute()

The slowest run took 6.01 times longer than the fastest. This could mean that an intermediate result is being cached.
10.3 s ± 3.39 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
